In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import sys
import numpy as np
import random
from utils import GroundTruthProcess
from utils import NCHW_to_NHWC_np
from utils import NHWC_to_NCHW_np
from DME_deformable import DMENet
from config import DefaultConfig
import torch.cuda as torch_cudab
import metrics
%matplotlib inline

In [2]:
# obtain the gpu device
assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU
config = DefaultConfig()

In [3]:
image_test_data = NHWC_to_NCHW_np(np.load(config.test_data_path))
gt_test_data = NHWC_to_NCHW_np(np.load(config.test_gt_path))
length = len(image_test_data)
mae_metrics = []
mse_metrics = []

In [4]:
net = torch.load(config.model_save_path).to(cuda_device)
net.eval()

gt_process_model = GroundTruthProcess(1, 1, 8).to(cuda_device)
ae_batch = metrics.AEBatch().to(cuda_device)
se_batch = metrics.SEBatch().to(cuda_device)

In [7]:
for i in range(length):
    if config.use_gpu:
        predict_x = torch.FloatTensor(image_test_data[i:i + 1]).cuda()
        predict_gt = torch.FloatTensor(gt_test_data[i:i + 1]).cuda()
    predict_predict_map = net(predict_x)
    predict_gt_map = gt_process_model(predict_gt)
    batch_ae = ae_batch(predict_predict_map, predict_gt_map).data.cpu().numpy()
    batch_se = se_batch(predict_predict_map, predict_gt_map).data.cpu().numpy()
    mae_metrics.append(batch_ae)
    mse_metrics.append(batch_se)
    # to numpy
    numpy_predict_map = predict_predict_map.data.cpu().numpy()
    numpy_gt_map = predict_gt_map.data.cpu().numpy()
    
    # show current prediction
    figure, (origin, dm_gt, dm_pred) = plt.subplots(1, 3, figsize=(20, 4))
    origin.imshow(np.squeeze(NCHW_to_NHWC_np(image_test_data[i:i+1])))
    origin.set_title('Origin Image')
    dm_gt.imshow(np.squeeze(NCHW_to_NHWC_np(numpy_gt_map)), cmap=plt.cm.jet)
    dm_gt.set_title('ground_truth')
    dm_pred.imshow(np.squeeze(NCHW_to_NHWC_np(numpy_predict_map)), cmap=plt.cm.jet)
    dm_pred.set_title('prediction')
    plt.suptitle('The ' + str(i + 1) + 'st images\'prediction')
    plt.show()
    sys.stdout.write('The grount truth crowd number is:{}, and the predicting number is:{}'.format(np.sum(numpy_gt_map), np.sum(numpy_predict_map)))
    sys.stdout.flush

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [6]:
mae_metrics = np.reshape(mae_metrics, [-1])
mse_metrics = np.reshape(mse_metrics, [-1])
MAE = np.mean(mae_metrics)
MSE = np.sqrt(np.mean(mse_metrics))
print('MAE:', MAE, 'MSE:', MSE)

MAE: 21.86331 MSE: 31.934145
